In [1]:
import cv2
import numpy as np
import imutils
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# resizing the input image, return resized version and ration to the original one
def resize_img(image):
    resized = imutils.resize(image, width=300)
    ratio = image.shape[0] / float(resized.shape[0])
    return resized, ratio

# preparing the image for HoughCircles algorithm to be applied
def prepare_img_for_circles(img):
    color = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    blurred = cv2.GaussianBlur(color, (5, 5), 3)
    color = cv2.cvtColor(blurred, cv2.COLOR_HSV2RGB)
    gray = cv2.cvtColor(color, cv2.COLOR_RGB2GRAY)
    thresh = cv2.threshold(gray, 135, 255, cv2.THRESH_TOZERO)[1]
    return thresh

In [3]:
# apply the HoughCircles algorithm and return found circles
def get_circles(img,
                minDist = 75,
                param1 = 60,
                param2 = 22, 
                minRadius = 4,
                maxRadius = 30):
    
    circles = cv2.HoughCircles(img,
                               cv2.HOUGH_GRADIENT,
                               1,
                               minDist, 
                               param1=param1, 
                               param2=param2, 
                               minRadius=minRadius, 
                               maxRadius=maxRadius)
    return circles

# display found circles in the source image (with given ratio)
def write_circles_on_img(circles, img, ratio, out_img):
    color = (255, 0, 0)
    if circles is not None:
        circles = np.uint16(np.around(circles*ratio))
        for circle in circles[0]:
            x = circle[0] - circle[2]
            y = circle[1] - circle[2]
            w = circle[2] * 2
            h = w
            draw_rect_text(out_img, "circle", x, y, w, h, color)
        

In [4]:
# preparing the image for finding contours
def prepare_img_for_angles(img):
    kernel = np.ones((2,2), np.uint8)
    
    color = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower = np.array([100, 59, 0],dtype='float')
    upper = np.array([255, 100, 255], dtype='float')
    mask = cv2.inRange(color, lower, upper)

    erosion = cv2.erode(mask, kernel, iterations=2)
    dilation = cv2.dilate(erosion, kernel, iterations=2)
    blurred = cv2.GaussianBlur(dilation, (1,3), 0)
    thresh = cv2.threshold(blurred, 175, 255, cv2.THRESH_BINARY)[1]
    return thresh

# drawing the rectangle over the found figure and put the text on top of it
def draw_rect_text(img, text, x, y, w, h, color, thickness = 2):
        cv2.rectangle(img, (x, y), (x + w, y + h), color, thickness)
        cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
        2, color, thickness)

# finding contours and checks if it rectangle or triangle
def find_figures(img, out_img, ratio):    
    cnts = cv2.findContours(img, cv2.RETR_LIST, cv2.CHAIN_APPROX_TC89_L1)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        tuple_out = cv2.boundingRect(c)
        arr = [int(x*ratio) for x in tuple_out]
        x, y, w, h = arr[0], arr[1], arr[2], arr[3]

        if len(approx) == 3 and peri > 50:
            color = (0,255,0)
            draw_rect_text(out_img, "triangle", x, y, w, h, color)
        elif len(approx) == 4 and peri > 50:
            color = (0,0,255)
            draw_rect_text(out_img, "rectangle", x, y, w, h, color)


In [5]:
# open the required video
cap = cv2.VideoCapture('Internship2021testtask.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(5))

# define the output source
out = cv2.VideoWriter('output_video.avi',cv2.VideoWriter_fourcc('M','J','P','G'),
                      fps, (frame_width,frame_height))

# perform the transformations for the each frame of the video and save it
while(True):
    
    ret, frame = cap.read()
    if ret == True:
        out_img = frame.copy()
        resized, ratio = resize_img(frame)
        thresh = prepare_img_for_circles(resized)
        circles = get_circles(thresh)
        write_circles_on_img(circles, resized, ratio, out_img)
        thresh = prepare_img_for_angles(resized)
        find_figures(thresh, out_img, ratio)
        out.write(out_img)
    else:
        break 

# releasing input and output stream
cap.release()
out.release()


<ipython-input-3-805e54c119c4>:26: RuntimeWarning: overflow encountered in ushort_scalars
  y = circle[1] - circle[2]
